In [2]:
import numpy as np
import pandas as pd
from alpha_vantage.timeseries import TimeSeries
import matplotlib.pyplot as plt
from datetime import date
from datetime import datetime
from datetime import time
from datetime import timedelta
from pandas.tseries.offsets import BDay
import matplotlib.lines as mlines
from dotenv import load_dotenv
import os
import matplotlib.patches as patches
import seaborn as sns
import warnings
import requests
import json
import pandas_ta as ta
from ta import trend, volatility, momentum, volume
import yfinance as yf
import time

from numpy import array
from importlib import reload # to reload modules if we made changes to them without restarting kernel


In [3]:
data_yf = pd.read_csv('data/all_stock_data.csv')
data_yf.isna().any().any()
data_yf

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,Sector,macd,...,WILLR_14_lag_3,WILLR_14_lag_4,WILLR_14_lag_5,OBV_lag_1,OBV_lag_2,OBV_lag_3,OBV_lag_4,OBV_lag_5,gain_loss_pct,win
0,2000-09-23,31.625000,32.937500,29.312500,30.156250,18.661280,441758600.0,MSFT,Technology,-3.148325,...,-80.370370,-60.778443,-54.444444,-3.000136e+09,-2.650566e+09,-2.336127e+09,-2.053990e+09,-1.774464e+09,-0.046443,-1
1,2000-09-30,30.250000,30.406250,27.250000,27.781250,17.191576,423283200.0,MSFT,Technology,-3.424853,...,-95.527157,-80.370370,-60.778443,-3.441894e+09,-3.000136e+09,-2.650566e+09,-2.336127e+09,-2.053990e+09,-0.081612,-1
2,2000-10-07,27.812500,28.468750,26.062500,26.875000,16.630775,416336400.0,MSFT,Technology,-3.674770,...,-89.714286,-95.527157,-80.370370,-3.865177e+09,-3.441894e+09,-3.000136e+09,-2.650566e+09,-2.336127e+09,-0.033708,-1
3,2000-10-14,26.750000,33.062500,24.218750,32.593750,20.169647,728943800.0,MSFT,Technology,-3.372500,...,-93.041237,-89.714286,-95.527157,-4.281514e+09,-3.865177e+09,-3.441894e+09,-3.000136e+09,-2.650566e+09,0.218458,1
4,2000-10-21,32.312500,34.593750,30.093750,33.843750,20.943165,686320600.0,MSFT,Technology,-2.997530,...,-96.255507,-93.041237,-89.714286,-3.552570e+09,-4.281514e+09,-3.865177e+09,-3.441894e+09,-3.000136e+09,0.047389,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
562609,2024-02-26,27.580000,28.059999,27.309999,27.400000,27.400000,3191000.0,NWS,Communication Services,1.691011,...,-22.121618,-9.001938,-5.587686,9.602336e+11,9.602313e+11,9.602277e+11,9.602222e+11,9.602266e+11,-0.006526,-1
562610,2024-03-04,27.469999,27.680000,26.600000,27.350000,27.350000,3164800.0,NWS,Communication Services,1.678421,...,-19.534286,-22.121618,-9.001938,9.602305e+11,9.602336e+11,9.602313e+11,9.602277e+11,9.602222e+11,-0.004368,-1
562611,2024-03-11,27.350000,27.910000,26.250000,26.440001,26.440001,5035500.0,NWS,Communication Services,1.576836,...,-18.926556,-19.534286,-22.121618,9.602273e+11,9.602305e+11,9.602336e+11,9.602313e+11,9.602277e+11,-0.033272,-1
562612,2024-03-18,26.570000,27.250000,26.250000,26.799999,26.799999,3302900.0,NWS,Communication Services,1.507996,...,-22.652761,-18.926556,-19.534286,9.602222e+11,9.602273e+11,9.602305e+11,9.602336e+11,9.602313e+11,0.008656,1


In [50]:
#working version 
import requests
import pandas as pd
import time

# take the top 300 stock tickers from our complete dataframe 
tickers = data_yf['Ticker'].unique()[:200]

# Replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
apikey = 'RNXMHV8CT56MK7AA'

# Set the time_from parameter to January 1, 2010, 12:00 AM
time_from = '20130101T0000'

# Get the current time in YYYYMMDDTHHMM format
time_to = '20230401T0000'

# Define the time interval between articles (1 week)
time_interval = 7 * 24 * 60 * 60  # seconds

# Function to fetch news sentiment data for a ticker within a time range
def fetch_news_sentiment(ticker, time_range):
    url = f'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={ticker}&apikey={apikey}&time_from={time_range[0]}&time_to={time_range[1]}&sort=LATEST&limit=1'
    r = requests.get(url)
    data = r.json()
    for item in data.get('feed', []):
        row = {
            "Ticker": ticker,
            "Time Published": item.get('time_published'),
            "Overall Sentiment Score": item.get('overall_sentiment_score'),
            "Ticker Sentiment Score": item.get('ticker_sentiment')[0].get('ticker_sentiment_score'),
            #"Relevance Score": item.get('topics')[0].get('relevance_score'),
            #"Ticker Sentiment Label": item.get('ticker_sentiment')[0].get('ticker_sentiment_label')
        }
        print(row)  # Print each row as it's created
        return row

# Function to split time range into smaller intervals
def split_time_range(time_from, time_to, interval):
    time_ranges = []
    current_time = int(time.mktime(time.strptime(time_to, '%Y%m%dT%H%M')))
    while current_time >= int(time.mktime(time.strptime(time_from, '%Y%m%dT%H%M'))):
        time_ranges.append((time.strftime("%Y%m%dT%H%M", time.gmtime(current_time - interval)), time.strftime("%Y%m%dT%H%M", time.gmtime(current_time))))
        current_time -= interval
    return time_ranges

# Split time range into smaller intervals
time_ranges = split_time_range(time_from, time_to, time_interval)

# Fetch news sentiment data for each ticker and time range
data_list = []
for time_range in time_ranges:
    for ticker in tickers:
        row = fetch_news_sentiment(ticker, time_range)
        if row:
            data_list.append(row)
            break  # Exit loop after fetching the first row for the week

# Create DataFrame
df = pd.DataFrame(data_list)
df.set_index('Time Published', inplace=True)
print(df)


{'Ticker': 'MSFT', 'Time Published': '20230401T043100', 'Overall Sentiment Score': 0.416008, 'Ticker Sentiment Score': '0.274586', 'Relevance Score': '0.161647', 'Ticker Sentiment Label': 'Somewhat-Bullish'}
{'Ticker': 'MSFT', 'Time Published': '20230325T060118', 'Overall Sentiment Score': -0.03676, 'Ticker Sentiment Score': '0.0', 'Relevance Score': '1.0', 'Ticker Sentiment Label': 'Neutral'}
{'Ticker': 'MSFT', 'Time Published': '20230318T050400', 'Overall Sentiment Score': 0.36848, 'Ticker Sentiment Score': '0.09307', 'Relevance Score': '1.0', 'Ticker Sentiment Label': 'Neutral'}
{'Ticker': 'MSFT', 'Time Published': '20230311T063300', 'Overall Sentiment Score': -0.040039, 'Ticker Sentiment Score': '-0.046094', 'Relevance Score': '1.0', 'Ticker Sentiment Label': 'Neutral'}
{'Ticker': 'MSFT', 'Time Published': '20230304T064800', 'Overall Sentiment Score': 0.246204, 'Ticker Sentiment Score': '0.2122', 'Relevance Score': '1.0', 'Ticker Sentiment Label': 'Somewhat-Bullish'}
{'Ticker': 'MS

KeyboardInterrupt: 

In [56]:
#changing granularity to monthly
import requests
import pandas as pd
import time
from datetime import datetime

# take the top 300 stock tickers from our complete dataframe 
tickers = data_yf['Ticker'].unique()[:300]

# Replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
apikey = 'RNXMHV8CT56MK7AA'

# Set the time_from parameter to January 1, 2010, 12:00 AM
time_from = '20100101T0000'

# Get the current time in YYYYMMDDTHHMM format
time_to = '20230401T0000'

# Function to fetch news sentiment data for a ticker within a time range
def fetch_news_sentiment(ticker, start_time, end_time):
    url = f'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={ticker}&apikey={apikey}&time_from={start_time}&time_to={end_time}&sort=EARLIEST&limit=1'
    r = requests.get(url)
    data = r.json()
    for item in data.get('feed', []):
        row = {
            "Ticker": ticker,
            "Time Published": item.get('time_published'),
            "Overall Sentiment Score": item.get('overall_sentiment_score'),
            "Ticker Sentiment Score": item.get('ticker_sentiment')[0].get('ticker_sentiment_score'),
            "Relevance Score": item.get('topics')[0].get('relevance_score'),
            #"Ticker Sentiment Label": item.get('ticker_sentiment')[0].get('ticker_sentiment_label')
        }
        print(row)  # Print each row as it's created
        return row

# Function to generate start and end times for the first day of each month
def generate_monthly_time_ranges(time_from, time_to):
    time_ranges = []
    current_time = datetime.strptime(time_to, '%Y%m%dT%H%M')
    while current_time >= datetime.strptime(time_from, '%Y%m%dT%H%M'):
        start_of_month = current_time.replace(day=1, hour=0, minute=0, second=0, microsecond=0)
        end_of_month = (start_of_month.replace(month=start_of_month.month % 12 + 1, year=start_of_month.year + (1 if start_of_month.month == 12 else 0))) - pd.Timedelta(days=1)
        time_ranges.append((start_of_month.strftime("%Y%m%dT%H%M"), end_of_month.strftime("%Y%m%dT%H%M")))
        current_time = start_of_month - pd.Timedelta(days=1)
    return time_ranges

# Generate start and end times for the first day of each month
time_ranges = generate_monthly_time_ranges(time_from, time_to)

# Fetch news sentiment data for each ticker and time range
data_list = []
for start_time, end_time in time_ranges:
    for ticker in tickers:
        row = fetch_news_sentiment(ticker, start_time, end_time)
        if row:
            data_list.append(row)
            break  # Exit loop after fetching the first row for the month

# Create DataFrame
df = pd.DataFrame(data_list)
df.set_index('Time Published', inplace=True)
print(df)


{'Ticker': 'MSFT', 'Time Published': '20230401T003414', 'Overall Sentiment Score': -0.095667, 'Ticker Sentiment Score': '-0.068007', 'Relevance Score': '0.451494'}
{'Ticker': 'MSFT', 'Time Published': '20230301T002401', 'Overall Sentiment Score': 0.252836, 'Ticker Sentiment Score': '0.091976', 'Relevance Score': '0.158519'}
{'Ticker': 'MSFT', 'Time Published': '20230201T000022', 'Overall Sentiment Score': 0.247546, 'Ticker Sentiment Score': '0.038657', 'Relevance Score': '0.897321'}
{'Ticker': 'MSFT', 'Time Published': '20230101T003005', 'Overall Sentiment Score': 0.12955, 'Ticker Sentiment Score': '0.163411', 'Relevance Score': '0.158519'}
{'Ticker': 'MSFT', 'Time Published': '20221201T004113', 'Overall Sentiment Score': 0.161181, 'Ticker Sentiment Score': '0.147128', 'Relevance Score': '0.158519'}
{'Ticker': 'MSFT', 'Time Published': '20221101T060000', 'Overall Sentiment Score': 0.305533, 'Ticker Sentiment Score': '0.0', 'Relevance Score': '0.5'}
{'Ticker': 'MSFT', 'Time Published': 